In [ ]:
import urllib.request
import re
import Data_Manager as dm

In [ ]:
liga = '3liga' # 2liga , 3liga, bundesliga
# dm.set_games()
# dm.drop_games()
stop = False
for year in range(2009,2020):
    Website = "https://www.fussballdaten.de/" + liga + "/" + str(year) + "/1/"
    try:
        w = urllib.request.urlopen(Website)
    except:
        print('URL unkown: ', Website)
        stop = True
    if stop:
        break
        
    if w.code == 200:
        html_cont = str(w.read())
        
    num_day = len(re.findall('option value.*?Spieltag', html_cont))
    print(Website, 'tage: ', num_day-1)
    del html_cont, w
    
    data = []
    
    for day in range(1,num_day):
        
        Webseite = "https://www.fussballdaten.de/" + liga + "/" + str(year) + "/" + str(day) + "/"
        try:
            w = urllib.request.urlopen(Webseite)
        except:
            print('URL unkown: ', Webseite)
            stop = True
        if stop:
            break
        
        if w.code == 200:
            html_cont = str(w.read())

        cutting = re.findall('leistungsdaten.*?</table>', html_cont)
        cutting = re.findall('<tr data-key.*?</tr>',cutting[0])
        
        for reihe in cutting:
            spalte = re.findall('<td.*?</td>',reihe)
            date = re.findall('b&gt;.*?&lt',spalte[0])[0][5:-3]
            team1 = re.findall('xs.*?<',spalte[2])[0][3:-1]
            if day == 1:
                team2 = re.findall('xs.*?<',spalte[4])[0][3:-1]
            else:
                team2 = re.findall('xs.*?<',spalte[4])[1][3:-1]
            ergebnis = re.findall('>.*?<',spalte[3])[2][1:-1]
            team1 = team1.replace('\\xc3\\xbc', 'ü')
            team1 = team1.replace('\\xc3\\xb6', 'ö')
            team1 = team1.replace('\\xc3\\x9f', 'ß')
            team2 = team2.replace('\\xc3\\xbc', 'ü')
            team2 = team2.replace('\\xc3\\xb6', 'ö')
            team2 = team2.replace('\\xc3\\x9f', 'ß')
            tore_heim = ergebnis.split(':')[0]
            tore_aus = ergebnis.split(':')[1]
            data.append((team1, tore_heim, tore_aus, team2, date, liga))
        
    dm.insert_games(data)